## Objective

This notebook focuses on extracting meaningful subgraphs from Panama Papers.
The goal is to isolate the most structurally and socially significant parts of the network — the nodes and relationships that reveal how offshore systems are organized.

### Mini-Dense Subgraph Extraction – Initial Node and Relationship Selection

#### Library Imports and Basic Configuration

In [1]:
import pandas as pd
import os
from tqdm import tqdm


DATA_DIR = "../Dataset"          # dossier d'origine
OUTPUT_DIR = "./extracts_mini_dense"  # dossier de sortie
os.makedirs(OUTPUT_DIR, exist_ok=True)

REL_FILE = os.path.join(DATA_DIR, "relationships.csv")
NODE_FILES = [
    os.path.join(DATA_DIR, "nodes-entities.csv"),
    os.path.join(DATA_DIR, "nodes-intermediaries.csv"),
    os.path.join(DATA_DIR, "nodes-officers.csv"),
    os.path.join(DATA_DIR, "nodes-addresses.csv")
]

#### Load Relationships


In [2]:
relationships = pd.read_csv(
    REL_FILE, 
    usecols=["node_id_start", "node_id_end", "rel_type"],
    low_memory=False
)
print(f"✅ Relationships loaded ({len(relationships):,} rows)")

✅ Relationships loaded (3,339,267 rows)


#### Identify Pivots

Pivots are highly connected or central nodes that act as key connectors within the network.

In [3]:
print("🔍 Calculating degree (number of connections)...")

deg = pd.concat([
    relationships["node_id_start"],
    relationships["node_id_end"]
]).value_counts().reset_index()
deg.columns = ["node_id", "degree"]

# 🔸 Adjust threshold here to target ~500–1000 nodes
threshold = 500  
top_nodes = deg[deg["degree"] > threshold]
print(f"✅ {len(top_nodes):,} pivots identified (degree > {threshold})")


🔍 Calculating degree (number of connections)...
✅ 339 pivots identified (degree > 500)


#### Extract Relationships Around Pivots

In [4]:
# 🧩 Extract relationships connected to the pivot nodes
print("🧩 Extracting sub-relations around pivots...")

sub_rels = relationships[
    relationships["node_id_start"].isin(top_nodes["node_id"]) |
    relationships["node_id_end"].isin(top_nodes["node_id"])
]

print(f"✅ {len(sub_rels):,} relationships extracted")


🧩 Extracting sub-relations around pivots...
✅ 616,008 relationships extracted


#### Extract corresponding nodes

In [5]:
# 📦 Extract nodes connected to the selected relationships
print("📦 Extracting related nodes...")

# Récupérer tous les node_id présents dans les relations extraites
node_ids = pd.unique(sub_rels[["node_id_start", "node_id_end"]].values.ravel())

all_nodes_list = []

# Charger les fichiers de nœuds et ne garder que ceux correspondant aux node_ids
for file in tqdm(NODE_FILES, desc="Loading nodes"):
    if os.path.exists(file):
        df = pd.read_csv(file, low_memory=False)
        df = df[df["node_id"].isin(node_ids)]
        all_nodes_list.append(df)

# Combiner tous les nœuds extraits en un seul DataFrame
sub_nodes = pd.concat(all_nodes_list, ignore_index=True)

print(f"✅ {len(sub_nodes):,} nodes extracted")


📦 Extracting related nodes...


Loading nodes: 100%|██████████| 4/4 [00:24<00:00,  6.04s/it]


✅ 463,389 nodes extracted


#### Add node type

In [6]:
sub_nodes["node_type"] = None
for file in NODE_FILES:
    if "entities" in file:
        sub_nodes.loc[sub_nodes["node_id"].isin(pd.read_csv(file)["node_id"]), "node_type"] = "Entity"
    elif "intermediaries" in file:
        sub_nodes.loc[sub_nodes["node_id"].isin(pd.read_csv(file)["node_id"]), "node_type"] = "Intermediary"
    elif "officers" in file:
        sub_nodes.loc[sub_nodes["node_id"].isin(pd.read_csv(file)["node_id"]), "node_type"] = "Officer"
    elif "addresses" in file:
        sub_nodes.loc[sub_nodes["node_id"].isin(pd.read_csv(file)["node_id"]), "node_type"] = "Address"

C:\Users\Ashahi\AppData\Local\Temp\ipykernel_24464\1968801901.py:4: DtypeWarning: Columns (2,3,6,7,10,11,12,13,14,15,16,17,20) have mixed types. Specify dtype option on import or set low_memory=False.
  sub_nodes.loc[sub_nodes["node_id"].isin(pd.read_csv(file)["node_id"]), "node_type"] = "Entity"


#### Save extracted nodes and relationships

In [7]:
sub_rels.to_csv(os.path.join(OUTPUT_DIR, "relations_extracted.csv"), index=False)
sub_nodes.to_csv(os.path.join(OUTPUT_DIR, "nodes_extracted.csv"), index=False)

print("\n🎉 Mini-Dense Extraction completed!")
print(f"👉 Nodes saved to: {OUTPUT_DIR}/nodes_extracted.csv")
print(f"👉 Relationships saved to: {OUTPUT_DIR}/relations_extracted.csv")


🎉 Mini-Dense Extraction completed!
👉 Nodes saved to: ./extracts_mini_dense/nodes_extracted.csv
👉 Relationships saved to: ./extracts_mini_dense/relations_extracted.csv


### Dense Subgraph Refinement – Filtering and Top Cluster Analysis


#### Library Imports and Basic Configuration

In [8]:
import pandas as pd
import networkx as nx
from tqdm import tqdm
import os

# Folder where the extracted files are located
extract_dir = "./extracts_mini_dense"

# Load CSVs
print("📂 Loading files...")
nodes = pd.read_csv(f"{extract_dir}/nodes_extracted.csv", low_memory=False)
rels = pd.read_csv(f"{extract_dir}/relations_extracted.csv", low_memory=False)
print(f"✅ {len(nodes):,} nodes | {len(rels):,} relationships")

📂 Loading files...
✅ 463,389 nodes | 616,008 relationships


#### Calculate Node Degree (number of connections)

In [9]:
# Calculate degree
degree = pd.concat([rels['node_id_start'], rels['node_id_end']]).value_counts()
degree = degree.rename_axis('node_id').reset_index(name='degree')

# Remove the 'degree' column if it already exists to avoid MergeError
if 'degree' in nodes.columns:
    nodes = nodes.drop(columns=['degree'])

# Merge with nodes
nodes = nodes.merge(degree, on='node_id', how='left')

# Replace NaN with 0
nodes['degree'] = nodes['degree'].fillna(0)

#### Filter “interesting” nodes

In [10]:
filtered_nodes = nodes[
    ((nodes["node_type"] == "Intermediary") & (nodes["degree"] > 50)) |
    ((nodes["node_type"] == "Officer") & (nodes["degree"] > 2)) |
    ((nodes["node_type"] == "Address") & (nodes["degree"] > 3)) |
    ((nodes["node_type"] == "Entity") & (nodes["degree"] > 1))
].copy()

# ✅ Use node_id instead of id
filtered_ids = set(filtered_nodes["node_id"])

filtered_rels = rels[
    rels["node_id_start"].isin(filtered_ids) &
    rels["node_id_end"].isin(filtered_ids)
].copy()

print(f"✅ Filtered nodes: {len(filtered_nodes):,}")
print(f"✅ Filtered relationships: {len(filtered_rels):,}")

✅ Filtered nodes: 93,695
✅ Filtered relationships: 244,795


#### Detect the densest clusters

In [11]:
G = nx.from_pandas_edgelist(filtered_rels, "node_id_start", "node_id_end")
components = sorted(nx.connected_components(G), key=len, reverse=True)

print(f"🔹 Number of subgraphs: {len(components)}")
print(f"🔹 Size of the largest cluster: {len(components[0])}")

# Keep the 5 largest clusters
top_components = components[:5]
top_nodes = set().union(*top_components)

# ✅ Corrected: node_id instead of id
dense_nodes = filtered_nodes[filtered_nodes["node_id"].isin(top_nodes)].copy()
dense_rels = filtered_rels[
    filtered_rels["node_id_start"].isin(top_nodes) &
    filtered_rels["node_id_end"].isin(top_nodes)
].copy()

print(f"🏆 Final graph: {len(dense_nodes):,} nodes | {len(dense_rels):,} relationships")

🔹 Number of subgraphs: 21
🔹 Size of the largest cluster: 57166
🏆 Final graph: 82,012 nodes | 218,457 relationships


#### Save the results

In [12]:
output_dir = f"{extract_dir}/filtered"
os.makedirs(output_dir, exist_ok=True)

dense_nodes.to_csv(f"{output_dir}/nodes_interesting.csv", index=False)
dense_rels.to_csv(f"{output_dir}/relationships_interesting.csv", index=False)

print("💾 Files saved in:", output_dir)
print("✅ Reduction successfully completed!")

💾 Files saved in: ./extracts_mini_dense/filtered
✅ Reduction successfully completed!


### Ultra-Targeted Mini-Graph Extraction – Top Hubs and Direct Neighbors

#### Basic configuration

In [13]:
# Paths
extract_dir = "./extracts_mini_dense/filtered"
output_dir = f"{extract_dir}/mini_graph"
os.makedirs(output_dir, exist_ok=True)

# Load filtered files
nodes = pd.read_csv(f"{extract_dir}/nodes_interesting.csv", low_memory=False)
rels = pd.read_csv(f"{extract_dir}/relationships_interesting.csv", low_memory=False)

#### Select top hubs

In [14]:
top_n = 500  # number of hubs to keep
top_hubs = nodes.sort_values("degree", ascending=False).head(top_n)
top_hub_ids = set(top_hubs["node_id"])

#### Extract direct neighbors

In [15]:
neighbors_start = rels[rels["node_id_start"].isin(top_hub_ids)]["node_id_end"]
neighbors_end = rels[rels["node_id_end"].isin(top_hub_ids)]["node_id_start"]

neighbor_ids = set(neighbors_start).union(set(neighbors_end))

# All nodes of the mini-graph
mini_node_ids = top_hub_ids.union(neighbor_ids)
mini_nodes = nodes[nodes["node_id"].isin(mini_node_ids)]

# Relationships of the mini-graph
mini_rels = rels[
    rels["node_id_start"].isin(mini_node_ids) &
    rels["node_id_end"].isin(mini_node_ids)
]

#### Save results

In [16]:
mini_nodes.to_csv(f"{output_dir}/nodes_mini.csv", index=False)
mini_rels.to_csv(f"{output_dir}/relationships_mini.csv", index=False)

print("✅ Ultra-targeted mini-graph created!")
print(f"📌 Nodes: {len(mini_nodes):,}")
print(f"📌 Relationships: {len(mini_rels):,}")
print(f"💾 Files saved in: {output_dir}")

✅ Ultra-targeted mini-graph created!
📌 Nodes: 82,012
📌 Relationships: 218,457
💾 Files saved in: ./extracts_mini_dense/filtered/mini_graph


### Final Subgraph Extraction & Metrics – Panama Papers

#### Loading the Mini-Graph

In [18]:
# Paths
output_dir = "./extracts_mini_dense/filtered/mini_graph"

# Load exported files
nodes = pd.read_csv(f"{output_dir}/nodes_mini.csv", low_memory=False)
rels = pd.read_csv(f"{output_dir}/relationships_mini.csv", low_memory=False)

# Create the graph
G = nx.from_pandas_edgelist(rels, "node_id_start", "node_id_end")

print("🔹 Number of nodes:", G.number_of_nodes())
print("🔹 Number of edges:", G.number_of_edges())
print("🔹 Number of connected components:", nx.number_connected_components(G))

# Largest cluster
largest_cc = max(nx.connected_components(G), key=len)
print("🔹 Size of the largest cluster:", len(largest_cc))

🔹 Number of nodes: 82012
🔹 Number of edges: 199012
🔹 Number of connected components: 5
🔹 Size of the largest cluster: 57166


#### Centrality Measures

In [19]:
degree_centrality = nx.degree_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G, k=2000)  # sample for speed

# Add to nodes dataframe
nodes["degree_centrality"] = nodes["node_id"].map(degree_centrality)
nodes["betweenness_centrality"] = nodes["node_id"].map(betweenness_centrality)

# Top 10 pivots by betweenness centrality
top_pivots = nodes.sort_values("betweenness_centrality", ascending=False).head(10)
print(top_pivots[["name", "node_type", "betweenness_centrality"]])

                                        name     node_type  \
81985                                    NaN       Address   
81906                Standard Directors Ltd.       Officer   
81907                        Execorp Limited       Officer   
49007              White Willow Trading Ltd.        Entity   
81914      Portcullis TrustNet (BVI) Limited       Officer   
81893  Christabel Corporate Services Limited  Intermediary   
81988                                    NaN       Address   
50835                 Wenbart Alliance Corp.        Entity   
53686               TRUDELT PRODUCTIONS INC.        Entity   
51272              SEPROTECH INVESTMENT INC.        Entity   

       betweenness_centrality  
81985                0.286665  
81906                0.205297  
81907                0.191670  
49007                0.190932  
81914                0.123408  
81893                0.097233  
81988                0.097233  
50835                0.061448  
53686                0.061448

#### Extract Narrative Subgraphs – Panama Papers

In [20]:
# Output folder
output_dir = "./subgraphs_narratives"
os.makedirs(output_dir, exist_ok=True)

print("🔍 Building main graph...")
G = nx.from_pandas_edgelist(rels, "node_id_start", "node_id_end")
G = G.subgraph(nodes["node_id"]).copy()  # ✅ correction

# Add node attributes from nodes dataframe
attrs = nodes.set_index("node_id").to_dict(orient="index")
nx.set_node_attributes(G, attrs)

# Check
print(f"📊 Global graph: {len(G):,} nodes | {G.number_of_edges():,} edges")

🔍 Building main graph...
📊 Global graph: 82,012 nodes | 199,012 edges


#### Extract: Central Intermediaries

In [21]:
# Select top central intermediaries
interm_nodes = nodes[nodes["node_type"] == "Intermediary"].nlargest(10, "degree_centrality")["node_id"]

# Build combined subgraph (radius=2 for neighbors)
sub1 = nx.ego_graph(G, interm_nodes.iloc[0], radius=2)
for n in interm_nodes[1:]:
    sub1 = nx.compose(sub1, nx.ego_graph(G, n, radius=2))

# Keep top 800 most connected nodes
degrees = dict(sub1.degree())
top_nodes = sorted(degrees, key=degrees.get, reverse=True)[:800]
sub1 = sub1.subgraph(top_nodes).copy()

print(f"📊 'Intermediaries' subgraph: {len(sub1)} nodes, {sub1.number_of_edges()} edges")

📊 'Intermediaries' subgraph: 800 nodes, 5348 edges


#### Pivot Officers Subgraph

In [22]:
# Select top connected officers
officers = nodes[nodes["node_type"] == "Officer"].nlargest(10, "degree_centrality")["node_id"]

# Build combined subgraph (merge ego_graphs)
sub3 = nx.ego_graph(G, officers.iloc[0], radius=2)
for o in officers[1:]:
    sub3 = nx.compose(sub3, nx.ego_graph(G, o, radius=2))

# Keep top 700 most connected nodes
degrees = dict(sub3.degree())
top_nodes = sorted(degrees, key=degrees.get, reverse=True)[:700]
sub3 = sub3.subgraph(top_nodes).copy()

print(f"📊 'Pivot Officers' subgraph: {len(sub3)} nodes, {sub3.number_of_edges()} edges")

📊 'Pivot Officers' subgraph: 700 nodes, 3279 edges


#### Export CSV for all subgraphs

In [23]:
def export_subgraph(subgraph, filename_base):
    """
    Export a NetworkX subgraph to CSV and GEXF
    """
    # Nodes with attributes
    nodes_df = pd.DataFrame.from_dict(dict(subgraph.nodes(data=True)), orient='index').reset_index()
    nodes_df.rename(columns={'index': 'node_id'}, inplace=True)
    
    # Edges / relationships
    edges_df = nx.to_pandas_edgelist(subgraph)
    
    # Export CSV
    nodes_df.to_csv(f"{output_dir}/{filename_base}_nodes.csv", index=False)
    edges_df.to_csv(f"{output_dir}/{filename_base}_edges.csv", index=False)
    
    print(f"✅ Subgraph '{filename_base}' exported: {len(nodes_df)} nodes, {len(edges_df)} edges")

# Exports
export_subgraph(sub1, "subgraph_1_intermediaries")
export_subgraph(sub3, "subgraph_3_officers")

✅ Subgraph 'subgraph_1_intermediaries' exported: 800 nodes, 5348 edges
✅ Subgraph 'subgraph_3_officers' exported: 700 nodes, 3279 edges


#### Basic Statistics of the Dataset

In [24]:
print("📊 Number of nodes:", G.number_of_nodes())
print("📊 Number of edges:", G.number_of_edges())
print("📊 Number of connected components:", nx.number_connected_components(G))


📊 Number of nodes: 82012
📊 Number of edges: 199012
📊 Number of connected components: 5
